# Notebook for finding duplicate data

TODO:

1) For each broker separately - look how much decrease...

2) Phase 2...

Following notebook is to find out how much duplicate requests.


With each request_uuid we have parameters - pickup_timetamp, return_timestamp, broker_contract, driver_age the same..

Duplicates can be spotted as following:

1) Same request_id

2) Different request_id, but close time, same broker and same other parameters (age, source...)



In [1]:
import pandas as pd
import csv
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (14, 6)
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [9]:
# Read data..

fields = ["timestamp", "pickup_timestamp", "return_timestamp", "broker_contract_id", "driver_age", 
          "request_uuid", "source_country_region_id", "pickup_desk_id", "return_desk_id"]
df = pd.read_csv("rate_quote.csv", skipinitialspace=True, usecols=fields)
print(df.shape)





(2997586, 9)


In [10]:
df["timestamp"] = pd.to_datetime(df['timestamp'])
df["pickup_timestamp"] = pd.to_datetime(df['pickup_timestamp'])
df["return_timestamp"] = pd.to_datetime(df['return_timestamp'])


# Merge with broker contract name
fields = ["Contract_ID", "Contract_name", "Broker name"]
contract_df = pd.read_csv("broker_contracts.csv", skipinitialspace=True, usecols=fields)
contract_df["Broker name"] = contract_df["Broker name"].astype(str)
contract_df["Contract_name"] = contract_df["Contract_name"].astype(str)
# This holds contract names and which broker name corresponds to each contract id..
#contract_df = contract_df.set_index("Contract_ID")

print(contract_df.shape)
print(contract_df.dtypes)


contract_df.columns = ["broker_contract_id", "Contract_name", "Broker_name"]
    
#print(df)
    
    
merged_requests = pd.merge(df, contract_df, on=["broker_contract_id"])

print(merged_requests.dtypes)

(27, 3)
Contract_ID       int64
Contract_name    object
Broker name      object
dtype: object
timestamp                   datetime64[ns]
pickup_desk_id                       int64
return_desk_id                       int64
pickup_timestamp            datetime64[ns]
return_timestamp            datetime64[ns]
broker_contract_id                   int64
source_country_region_id             int64
driver_age                           int64
request_uuid                        object
Contract_name                       object
Broker_name                         object
dtype: object


## Phase 1 - Removing duplicates using request_uuid

1) Load in all data

2) Group data by request_uuid, for each find minimal timestamp and set it as aggregate row's timestamp. We can groupby and get first of each (we make the assumption that the data is in chronological order)..

In [29]:
first_from_each_requestuuid = merged_requests[merged_requests.groupby('request_uuid')['timestamp'].rank() == 1 ]


In [31]:
len(first_from_each_requestuuid)

130915

### How many removed?

Before: 2997586 After: 130915
Decrease of 95.63265240763735%

We see that there are A LOT of duplicates based already only on request_uuid

In [41]:
before_count = len(merged_requests)
after_count =  len(first_from_each_requestuuid)

print("Before:", before_count, "After:", after_count)
print("Decrease of {}%".format((100-(after_count/before_count)*100)))



Before: 2997586 After: 130915
Decrease of 95.63265240763735%


In [42]:
## Saving data frame (without duplicates by requestuuid)

first_from_each_requestuuid.to_csv("rate_quote_1_dup1.csv")

## Phase 2 - Removing extra duplicates - different request_uuid, but other parameters same (nearby timestamp) 